In [3]:
exec(open('libraries.py').read())
cliente_tabla = pd.read_csv('data/cliente_tabla.csv')
producto_tabla = pd.read_csv('data/producto_tabla.csv')
test = pd.read_csv('data/test.csv')
train = pd.read_csv('data/train.csv')
town_state = pd.read_csv('data/town_state.csv')

# Benchmark model on holdout set

Just take the average of the sales of the product in the shop<br>
Test the holdout accuracy by first using this simple model to predict week 8 and 9

In [4]:
holdout_test = train.loc[train['Semana'] > 7,:]
holdout_train = train.loc[train['Semana'] <= 7,:]
print('Size holdout train: %s' %str(holdout_train.shape))
print('Size holdout test: %s' %str(holdout_test.shape))
holdout_train.sample(20)

Size holdout train: (53364883, 11)
Size holdout test: (20815581, 11)


,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Venta_uni_hoy,Venta_hoy,Dev_uni_proxima,Dev_proxima,Demanda_uni_equil
8690062,3,2054,1,1026,569439,73,1,21.32,1,21.32,0
36086177,6,1349,1,2126,8243516,32934,2,14.08,0,0.00,2
16640509,4,1612,1,2112,489439,31310,24,150.00,0,0.00,24
5341726,3,1555,1,1026,181931,36287,7,80.78,0,0.00,7
8194623,3,2029,1,1129,229905,35651,4,30.00,0,0.00,4
17699986,4,1911,1,1068,4359560,3631,4,65.40,0,0.00,4
8628322,3,2044,1,1235,1846705,1064,3,50.01,0,0.00,3
27111205,5,1549,1,2034,1386477,5328,2,17.78,0,0.00,2
52290433,7,3212,1,1226,259646,1242,7,53.48,0,0.00,7
53167254,7,22560,1,2022,2071731,5355,4,20.76,0,0.00,4


#### Create lookup table, grouped by client and product

In [5]:
grouped_by_prod_client = holdout_train[['Cliente_ID', 'Producto_ID', 'Demanda_uni_equil']]\
                                            .groupby(['Cliente_ID', 'Producto_ID'])
grouped_by_prod_client = grouped_by_prod_client.aggregate('mean')\
                                        .rename(columns={'Demanda_uni_equil': 'Lookup_demand_Prod_Client'})
grouped_by_prod_client.head()

Lookup_demand_Prod_Client
Cliente_ID Producto_ID                           
26         1182                              19.5
           4767                              42.0
           30314                             48.0
           31393                             16.8
           31518                             10.0

In [7]:
median_demanda = holdout_train['Demanda_uni_equil'].median()
median_demanda

3.0

#### Join lookup tables with test data to form predictions

In [18]:
holdout_test_merge = pd.merge(holdout_test, grouped_by_prod_client, how='left',
         left_on=['Cliente_ID', 'Producto_ID'], right_index=True)
holdout_test_merge.head()

,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Venta_uni_hoy,Venta_hoy,Dev_uni_proxima,Dev_proxima,Demanda_uni_equil,Lookup_demand_Prod_Client,Lookup_demand_Prod,Lookup_demand_Client
53364883,8,1110,7,3301,15766,1212,4,33.52,0,0.0,4,4.000000,2.907786,4.0
53364884,8,1110,7,3301,15766,1216,5,41.90,0,0.0,5,2.400000,3.110175,4.0
53364885,8,1110,7,3301,15766,1220,1,7.64,0,0.0,1,3.000000,3.168675,4.0
53364886,8,1110,7,3301,15766,1238,3,29.49,0,0.0,3,2.400000,3.223185,4.0
53364887,8,1110,7,3301,15766,1240,2,16.76,0,0.0,2,6.333333,5.725000,4.0


In [62]:
percent_not_predicted = sum(holdout_test_merge['Lookup_demand_Prod_Client'].isnull())/ \
                                float(len(holdout_test_merge['Lookup_demand_Prod_Client']))
print('Percent not predicted with lookup table Client - Product: %f' %percent_not_predicted)

Percent not predicted with lookup table Client - Product: 0.219095
Percent not predicted with lookup table Product: 0.000423


#### Predict with these lookup tables:

- If the client and product is in the lookup table Lookup_demand_Prod_Client, use this value
- If not, use median

In [27]:
holdout_test_pred = holdout_test_merge.copy()
holdout_test_pred['Prediction'] = median_demanda
holdout_test_pred.loc[holdout_test_pred['Lookup_demand_Prod_Client'].notnull(),'Prediction'] = \
    holdout_test_pred.loc[holdout_test_pred['Lookup_demand_Prod_Client'].notnull(),'Lookup_demand_Prod_Client']

#### Asses goodness of fit week 8

In [28]:
week_8 = holdout_test_pred.loc[holdout_test_pred['Semana']==8,:]
temp = np.log(week_8['Prediction'] + 1) - np.log(week_8['Demanda_uni_equil'] + 1)
print('Goodness of fit week 8: %f' %np.sqrt(np.sum(np.square(temp))/len(temp)))

Goodness of fit week 8: 0.539437


#### Asses goodness of fit week 9

In [29]:
week_9 = holdout_test_pred.loc[holdout_test_pred['Semana']==9,:]
temp = np.log(week_9['Prediction'] + 1) - np.log(week_9['Demanda_uni_equil'] + 1)
print('Goodness of fit week 9: %f' %np.sqrt(np.sum(np.square(temp))/len(temp)))

Goodness of fit week 9: 0.562664


# Same benchmark model on test set

In [47]:
grouped_by_client = train[['Cliente_ID', 'Producto_ID', 'Demanda_uni_equil']]\
                                            .groupby(['Cliente_ID', 'Producto_ID'])
grouped_by_client = grouped_by_client.aggregate('mean').rename(columns={'Demanda_uni_equil': 'Lookup_demand_Prod_Client'})
grouped_by_client.head()

Lookup_demand_Prod_Client
Cliente_ID Producto_ID                           
26         1182                              13.0
           4767                              42.0
           30235                             96.0
           30314                             48.0
           31393                             17.0

In [50]:
test_merge = pd.merge(test, grouped_by_client, how='left',
         left_on=['Cliente_ID', 'Producto_ID'], right_index=True)
test_merge.head()

,id,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Lookup_demand_Prod_Client,Lookup_demand_Prod
0,0,11,4037,1,2209,4639078,35305,4.0,9.182930
1,1,11,2237,1,1226,4705135,1238,NaN,3.175030
2,2,10,2045,1,2831,4549769,32940,2.0,4.008649
3,3,11,1227,1,4448,4717855,43066,NaN,2.436524
4,4,11,1219,1,1130,966351,1277,NaN,3.411765


In [52]:
test_pred = test_merge
median_demanda = train['Demanda_uni_equil'].median()
test_pred['Prediction'] = median_demanda
test_pred.loc[test_pred['Lookup_demand_Prod_Client'].notnull(),'Prediction'] = \
    test_pred.loc[test_pred['Lookup_demand_Prod_Client'].notnull(),'Lookup_demand_Prod_Client']

In [54]:
submission = test_pred[['id', 'Prediction']]
submission = submission.rename(columns = {'Prediction':'Demanda_uni_equil'})
submission.head()

,id,Demanda_uni_equil
0,0,4.000000
1,1,3.175030
2,2,2.000000
3,3,2.436524
4,4,3.411765


In [55]:
submission.to_csv('submissions/benchmark_submission.csv', index = False)

# Train holdout score versus submission

Score with holdout: 0.539 <br>
Score on Kaggle: 0.530 <br>